## Code for fetching the below mentioned attributes from worldcat.org website

- Page URL of the book
- OCLC number of the book
- More books by the author

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import pickle
import time

In [6]:
driver = webdriver.Chrome(executable_path='/home/pybeast/chromedriver')

In [3]:
p = pickle.load(open('/home/pybeast/Desktop/IndicWiki/BooksProject/books_v6.pkl', "rb"))
isbn_list = p['isbn_13'].tolist()

In [4]:
def isBook(element):
    content = element.text
    content = content.replace("\n", "")
    content = content.replace(" ", "")
    if content == '(Book)':
        return True
    return False

In [5]:
# Accept cookies for first window
driver.get("https://www.worldcat.org")
time.sleep(10)
try:
    cookies = driver.find_element_by_id("onetrust-accept-btn-handler")
    cookies.click()
except:
    pass

d = dict()
cnt = 5500
for isbn in isbn_list[5500:]:
    status_str = ""
    cnt += 1
    isbn = str(isbn)
    url = "https://www.worldcat.org/isbn/{}"
    driver.get(url.format(isbn))
    time.sleep(1)
    
    # Empty dictionary for each book
    d[isbn] = dict()
    
    # Page url
    try:
        book_url = driver.current_url
        status_str += "Done fetching URL, "
    except:
        book_url = ""
        status_str += "Failed fetching URL, "
    
    # OCLC number of book(An identfier found in most articles of Wikipedia on books)
    try:
        oclc_number = driver.find_element_by_id("details-oclcno").find_element_by_tag_name("td").text
        status_str += "Done fetching OCLC, "
    except:
        oclc_number = ""
        status_str += "Failed fetching OCLC, "
    
    # Some more works by the author
    try:
        more_works = list()
        driver.find_element_by_id("details-allauthors").find_element_by_tag_name("input").click()
        time.sleep(2)
        html_code = driver.find_element_by_id("identitiesworksby").get_attribute("innerHTML")
        soup = BeautifulSoup(html_code, "html.parser")
        all_divs = soup.find_all(class_="identitiesWorksByEntry")
        for div in all_divs:
            element = div.find('span', class_="identitiesformat")
            if isBook(element):
                book_title = div.find('a', {'target': '_top'}).text
                more_works.append(book_title)
        status_str += "Done fetching MORE WORKS"
    except:
        more_works = list()
        status_str += "Failed fetching MORE WORKS"
    
    # Adding items to the dictionary
    d[isbn]['book_url'] = book_url
    d[isbn]['oclc_number'] = oclc_number
    d[isbn]['more_works'] = more_works
    
    # Status
    print("At #" + str(cnt) + " " + status_str)
    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.72)


In [126]:
f = open("/home/pybeast/Desktop/IndicWiki/BooksProject/worldcat_dot_org/801_1800.txt", "w")
f.write(str(d))
f.close()

In [125]:
len(d)

2330

In [110]:
driver.quit()